## Deliverable 2. Create a Customer Travel Destinations Map.

In [174]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [175]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('C:/Users/Lakenia/Desktop/World_Weather_Analysis/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,rikitea,PF,-23.12,-134.97,74.46,86,86,21.72,light rain
1,1,atuona,PF,-9.80,-139.03,77.41,80,28,14.34,light rain
2,2,kapaa,US,22.08,-159.32,82.40,74,1,5.82,clear sky
3,3,bredasdorp,ZA,-34.53,20.04,64.40,88,39,8.05,scattered clouds
4,4,borger,NL,52.92,6.79,62.01,93,98,11.01,overcast clouds


In [176]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_prompt_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_prompt_temp = float(input('What is the maximum temperature you would like for your trip? '))                    



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


In [177]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_prompt_temp) & \
                                       (city_data_df["Max Temp"] >= min_prompt_temp)]

preferred_cities_df.head(10)

len(preferred_cities_df)
        

85

In [178]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False
14,False,False,False,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False,False,False,False
19,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
532,False,False,False,False,False,False,False,False,False,False
538,False,False,False,False,False,False,False,False,False,False
540,False,False,False,False,False,False,False,False,False,False
551,False,False,False,False,False,False,False,False,False,False


In [179]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()


In [180]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,atuona,PF,77.41,light rain,-9.80,-139.03,
7,vaini,TO,78.80,few clouds,-21.20,-175.20,
14,puerto ayora,EC,77.00,scattered clouds,-0.74,-90.35,
15,eyl,SO,79.09,clear sky,7.98,49.82,
19,morondava,MG,75.79,clear sky,-20.28,44.28,
27,avarua,CK,78.80,broken clouds,-21.21,-159.78,
33,port-gentil,GA,78.80,thunderstorm,-0.72,8.78,
45,hambantota,LK,77.00,scattered clouds,6.12,81.12,
58,bandarbeyla,SO,76.80,clear sky,9.49,50.81,
63,comodoro rivadavia,AR,78.80,clear sky,-45.87,-67.50,


In [181]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping..zz")
        
len(hotels["results"])
        

Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz
Hotel not found... skipping..zz


9

In [182]:
# 7. Drop the rows where there is no Hotel Name.

hotel_clean_df = hotel_df[hotel_df["Hotel Name"] != "" ]




hotel_clean_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,atuona,PF,77.41,light rain,-9.80,-139.03,Villa Enata
7,vaini,TO,78.80,few clouds,-21.20,-175.20,Keleti Beach Resort
14,puerto ayora,EC,77.00,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
15,eyl,SO,79.09,clear sky,7.98,49.82,Amiin Hotel
19,morondava,MG,75.79,clear sky,-20.28,44.28,La Case Bambou
...,...,...,...,...,...,...,...
527,mangrol,IN,78.51,scattered clouds,21.12,70.12,Hotel The Royal Honours
538,san quintin,MX,76.42,clear sky,30.48,-115.95,Old Mill Hotel
540,takoradi,GH,78.66,overcast clouds,4.88,-1.76,Raybow International Hotel
551,santo antonio do ica,BR,79.21,broken clouds,-3.10,-67.94,Hotel Fogaça


In [186]:
# 8a. Create the output File (CSV)
hotel_clean_df = pd.DataFrame(clean_hotel_clean_df)

# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(path_or_buf='C:/Users/Lakenia/Desktop/World_Weather_Analysis/World_Weather_Analysis/Vacation_Search/WeatherPy_Vacation.csv', index_label="City_ID")

In [187]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

In [188]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))